# Recuperação de Processos
É descrito aqui o modelo de recuperação dos processos e seus dados para população do banco.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import requests, os, re, json
from PyPDF2 import PdfReader
from time import sleep

## Coleta de nomes de advogados
Para obter os processos, é necessária uma chave de busca nos portais de consulta processual do país. Houve uma tentativa inicial, tentando gerar de forma aleatória várias códigos CNJ, mas essa implementação se mostrou custosa e pouco eficiente. 

Observando os processos, foi fácil ver que o ponto comum entre todos os processos é a presença de um advogado ou promotor de justiça. Dessa forma, o nome dos advogados se tornou chave para busca de processos, tornando necessária a listagem de advogados por cada estado.

Assim, iniciou o processo de coleta dos nomes dos advogados a partir dos resultados dos exames da OAB.

In [2]:
driver = webdriver.Firefox()
driver.get("https://oab.fgv.br/")

Inicialmente, os links para a página dos resultados são listados e acessados sequencialmente, seguido do download do último arquivo PDF disponível.

In [5]:
links = driver.find_elements(By.CSS_SELECTOR, "ul li a")
links = [link.get_attribute('href') for link in links]

for link in links:
    driver.get(link)

    select = Select(driver.find_element(By.CSS_SELECTOR, "select"))
    select.select_by_visible_text('OAB / RJ')

    files_list = driver.find_elements(By.CSS_SELECTOR, "table tr a")

    if len(files_list) == 0:
        continue

    files_list = [file.get_attribute("href") for file in files_list]
    files_list = [file for file in files_list if file.endswith('.pdf')]

    filename = files_list[0].split('/')[-1]
    response = requests.get(files_list[0])
    open(f"oab_pdfs/{filename}", "wb").write(response.content)

Após o download dos arquivos em PDF, os textos são extraídos tratados e separados utilizando REGEX e alocados em um dicionário que separa os advogados por estado.

In [87]:
pdf_path = "./oab_pdfs/"
pdf_list = os.listdir(pdf_path)

names = {}

for pdf_file in pdf_list:
    pdf_obj = open(pdf_path+pdf_file, 'rb')
    pdf_reader = PdfReader(pdf_obj)

    all_text = ""
    for i in range(pdf_reader.numPages):
        page_obj = pdf_reader.getPage(i)
        text = page_obj.extractText()
        all_text += (text)
        if all_text[-1] == ' ':
            all_text = all_text[:-2]

    all_text = all_text.replace("\n", "")
    state_marks = re.findall("\d*. OAB / \w{2}", all_text)
    state_marks = [re.sub("\d*. OAB /", "", state) for state in state_marks]
    
    divisions = re.split("\d*. OAB / \w{2}", all_text)
    for i, division in enumerate(divisions):
        division = re.sub("\d.\d. [A-Z] ", "", division)
        division = division.split("/")
        for l, item in enumerate(division):
            item = item.split(",")[-1]
            item = item.replace(".", "")
            item = item.replace("  ", " ")
            division[l] = item.strip()
        divisions[i] = division

    del divisions[0]

    for i, state in enumerate(state_marks):
        state = state.strip()
        if state not in names:
            names[state] = []
        names[state] += divisions[i]

Como é possível ver, o resgaste dos nomes dos advogados é bastante demorado. Por isso, o dicionário foi exportado para um arquivo json, para reduzir tempo de futuras coletas.

In [92]:
with open("names.json", "w") as output_file:
    json.dump(names, output_file)

Após a recuperação dos dados do arquivo JSON, incia-se a recuperação dos processos

In [3]:
with open("names.json") as file:
    names = json.load(file)

## Coleta dos processos
Agora, os processos são coletados sequencialmente a partir de buscas no site do PJE do Ceará. São realizadas buscas pelo nome do advogado, e o resultados são coletados e armazenados em um lista de dicionários. Esses dicionários registram:
- Número do processo
- Data
- Jurisdição
- Classe judicial
- Assunto do processo
- Órgão julgador
- Participantes do Polo Ativo
- Participantes do Polo Passivo
- Lista de movimentações dos processos

In [40]:
driver = webdriver.Firefox()
driver.get("https://pje.tjce.jus.br/pje1grau/ConsultaPublica/listView.seam")

base_path = "https://pje.tjce.jus.br"

input = driver.find_element(By.ID, "fPP:j_id165:nomeAdv")
submit_btn = driver.find_element(By.ID, "fPP:searchProcessos")

processos = []

def parse_td(td):
    title = td.find_element(By.CLASS_NAME, "name")
    title = title.text.strip()
    content = td.find_element(By.CLASS_NAME, "value")
    content = content.text.strip() 
    return (title, content)

def parse_tr(tr):
    tds = tr.find_elements(By.TAG_NAME, "td")
    nome = tds[0]
    nome = nome.text.strip()
    status = tds[1]
    status = status.text.strip()
    return {'nome': nome, 'status': status}

for name in names['CE'][-1:0:-1]:
    driver.get("https://pje.tjce.jus.br/pje1grau/ConsultaPublica/listView.seam")
    input = driver.find_element(By.ID, "fPP:j_id165:nomeAdv")
    submit_btn = driver.find_element(By.ID, "fPP:searchProcessos")

    name = name.replace("  ", " ")
    input.clear()
    input.send_keys(name)
    submit_btn.click()

    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.invisibility_of_element_located((By.ID, '_viewRoot:status.start')))
    except:
        continue

    table = driver.find_element(By.ID, "fPP:processosTable:tb")
    results = table.find_elements(By.CSS_SELECTOR, "tr td a")

    if len(results) == 0:
        continue

    results = [result.get_attribute('onclick') for result in results]
    results = [result[:-2].replace("openPopUp('Consulta pública','", "") for result in results]
    results = list(dict.fromkeys(results))

    for result in results:
        processo = {
            'numero': None,
            'data': None,
            'jurisdicao': None,
            'classe_judicial': None,
            'assunto' : [],
            'orgao_julgador': None,
            'polo_ativo': [],
            'polo_passivo': [],
            'movimentacao': [],
        }

        driver.get(base_path + result)
        general_data = driver.find_element(By.ID, "j_id131")
        general_data = general_data.find_elements(By.TAG_NAME, "td")
        general_data = dict([parse_td(td) for td in general_data])
        
        processo['numero'] = general_data['Número Processo'].strip()
        processo['data'] = general_data['Data da Distribuição'].strip()
        processo['jurisdicao'] = general_data['Jurisdição'].strip()
        processo['classe_judicial'] = general_data['Classe Judicial'].strip()
        processo['assunto'] = general_data['Assunto'].strip()
        processo['orgao_julgador'] = general_data['Órgão Julgador'].strip()

        polo_ativo = driver.find_element(By.ID, "j_id131:processoPartesPoloAtivoResumidoList:tb")
        polo_ativo = polo_ativo.find_elements(By.CSS_SELECTOR, "tbody tr")
        polo_ativo = [parse_tr(tr) for tr in polo_ativo]
        processo['polo_ativo'] = polo_ativo

        polo_passivo = driver.find_element(By.ID, "j_id131:processoPartesPoloPassivoResumidoList:tb")
        polo_passivo = polo_passivo.find_elements(By.CSS_SELECTOR, "table tbody tr")
        polo_passivo = [parse_tr(tr) for tr in polo_passivo]
        processo['polo_passivo'] = polo_passivo

        movimentacao = driver.find_element(By.ID, "j_id131:processoEventoPanel")
        movimentacao = movimentacao.find_elements(By.CSS_SELECTOR, "tbody td:nth-child(1)")
        movimentacao = [td.text.strip() for td in movimentacao if len(td.text.strip()) > 0]
        processo['movimentacao'] = movimentacao

        processos.append(processo)
